In [1]:
## BIBLIOTECAS
import requests
from bs4 import BeautifulSoup

#Manipulação de Dados
import pandas as pd
import numpy as np

import shutil
import os
import pyautogui

import unidecode #arrumar textos

#tempo
import time
from time import sleep

import re #regex

import matplotlib.pyplot as plt
import matplotlib.cm as cm

import matplotlib.font_manager as fm #para alterar as fontes dos graficos
import seaborn as sns

import pyperclip

import statistics
import math

from scipy import stats

import warnings #Warning inuteis
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)


# Definindo o estilo de fonte e tema
fonte = fm.FontProperties(family="Calibri", style="italic")

sns.set_style('whitegrid') #cor da grade(fundo)

import pyperclip
from tableone import TableOne

#TESTE ANOVA
import statsmodels.api as sm
from statsmodels.formula.api import ols

import seaborn

from statsmodels.stats.stattools import durbin_watson #teste de autocorrelação

In [2]:
inicio = time.time() #iniciando o cronometro

df = pd.read_excel("0 - Dados Limpos.xlsx")
df_bruto = df.copy()
df.sample(5)

fim = time.time() #encerrando cronometro
print('Tempo para rodar a simulação:', round(fim - inicio, 3))

Tempo para rodar a simulação: 160.004


In [3]:
df.columns

Index(['Ano', 'Unidade',
       'Qual é sua principal área ou unidade neste hospital?',
       '1. As pessoas se apoiam umas às outras nesta unidade',
       '2. Temos pessoas suficientes para lidar com o volume de trabalho',
       '3. Quando há muito trabalho a ser feito e rapidamente, trabalhamos juntos em equipe para realizar a tarefa',
       '4. Nesta unidade, as pessoas se tratam com respeito',
       '5. Os funcionários desta unidade trabalham mais horas do que o recomendado no atendimento a pacientes',
       '6. Estamos ativamente buscando melhorias para a segurança do paciente',
       '7. Utilizamos mais funcionários temporários do que o recomendado no atendimento a paciente',
       '8. Os funcionários sentem que seus erros são usados contra eles',
       '9. Erros que ocorreram levaram a mudanças positivas nesta unidade',
       '10. É apenas por acaso que erros mais sérios não acontecem por aqui',
       '11. Quando uma área nesta unidade fica muito movimentada, as demai

In [4]:
for coluna in df.columns:
    print(coluna, '='*25)
    counts = df[coluna].value_counts()
    percentages = df[coluna].value_counts(normalize=True).mul(100).round(1).astype(str) + '%'

    # Usando o método 'agg' para juntar contagens e porcentagens em uma mesma DataFrame
    res = pd.DataFrame({'counts': counts, 'percentage': percentages})
    print(res)
    print('\n')

Ano =========================
      counts percentage
2022   48176      18.6%
2021   43317      16.7%
2020   36945      14.2%
2019   32631      12.6%
2018   27828      10.7%
2017   26698      10.3%
2016   23506       9.1%
2015   13318       5.1%
2014    6849       2.6%


Unidade =========================
               counts percentage
Itaim           13456       5.2%
Copa D'Or       10705       4.1%
Quinta D'Or     10309       4.0%
Brasil           9982       3.9%
Anália Franco    9756       3.8%
...               ...        ...
Yutaka            296       0.1%
Santa Emília      200       0.1%
HNSN Epitácio     198       0.1%
Santa Marina      122       0.0%
Orthoservice       87       0.0%

[71 rows x 2 columns]


Qual é sua principal área ou unidade neste hospital? =========================
                                          counts percentage
Unidade de terapia intensiva adulto        34413      15.3%
Unidade de Internação adulto               31293      13.9%
Administrativa

     counts percentage
1.0  136450      53.4%
0.0  119197      46.6%


35. Existe uma boa cooperação entre as unidades hospitalares que precisam trabalhar juntas =========================
     counts percentage
1.0  158040      61.7%
0.0   98110      38.3%


36. Importantes informações sobre a assistência se perdem durante as mudanças de turno =========================
     counts percentage
1.0  130810      51.2%
0.0  124896      48.8%


37. Muitas vezes é desagradável trabalhar unidades com funcionários de outras unidades do hospital =========================
     counts percentage
1.0  144136      56.3%
0.0  111781      43.7%


38. Os problemas com frequência ocorrem na troca de informações entre as unidades do hospital =========================
     counts percentage
0.0  133429      52.2%
1.0  121999      47.8%


39. As ações da administração do hospital mostram que a segurança do paciente é uma prioridade máxima =========================
     counts percentage
1.0  206676      80

   counts percentage
0  193151      74.5%
1   66117      25.5%


6. Há quanto tempo você trabalha em sua profissão ou especialização atual?_3. 11 a 15 anos =========================
   counts percentage
0  220897      85.2%
1   38371      14.8%


6. Há quanto tempo você trabalha em sua profissão ou especialização atual?_4. 16 a 20 anos =========================
   counts percentage
0  237329      91.5%
1   21939       8.5%


6. Há quanto tempo você trabalha em sua profissão ou especialização atual?_5. 21 anos ou mais =========================
   counts percentage
0  239745      92.5%
1   19523       7.5%




# Contagens Simples

# Contagens por grupo (Acreditação)

In [5]:
lista_scores = ['1. As pessoas se apoiam umas às outras nesta unidade',
                '2. Temos pessoas suficientes para lidar com o volume de trabalho',
                '3. Quando há muito trabalho a ser feito e rapidamente, trabalhamos juntos em equipe para realizar a tarefa',
                '4. Nesta unidade, as pessoas se tratam com respeito',
                '5. Os funcionários desta unidade trabalham mais horas do que o recomendado no atendimento a pacientes',
                '6. Estamos ativamente buscando melhorias para a segurança do paciente',
                '7. Utilizamos mais funcionários temporários do que o recomendado no atendimento a paciente',
                '8. Os funcionários sentem que seus erros são usados contra eles',
                '9. Erros que ocorreram levaram a mudanças positivas nesta unidade',
                '10. É apenas por acaso que erros mais sérios não acontecem por aqui',
                '11. Quando uma área nesta unidade fica muito movimentada, as demais ajudam',
                '12. Quando um evento é notificado, sentimos que o foco se concentra no indivíduo, e não no problema',
                '13. Quando fazemos mudanças para melhorar a segurança do paciente, nós avaliamos sua efetividade',
                '14. Trabalhamos “em modo de crise”, tentando fazer coisas demais, rápido demais',
                '15. A segurança do paciente nunca é sacrificada em prol de se trabalhar mais',
                '16. Os funcionários se preocupam que seus erros sejam registrados em seu arquivo funcional',
                '17. Temos problemas com a segurança do paciente nesta unidade',
                '18. Nossos procedimentos e sistemas são bons para impedir que os erros aconteçam',
                '19. Meu supervisor/gerente elogia quando vê um trabalho feito de acordo com os procedimentos estabelecidos para a segurança do paciente.',
                '20. Meu supervisor/gerente considera seriamente as sugestões dos funcionários para melhorar a segurança do paciente',
                '21. Sempre que a pressão aumenta, meu supervisor/gerente quer que trabalhemos mais rápido, mesmo que isto signifique tomar atalhos',
                '22. Meu supervisor/gerente ignora problemas recorrentes na segurança do paciente',
                '23. Recebemos feedback das mudanças implementadas com base nos eventos notificados',
                '24. Os funcionários falam voluntariamente se vêem algo que possa afetar negativamente o atendimento aos pacientes',
                '25. Somos informados sobre os erros que acontecem nesta unidade',
                '26. Os funcionários sentem-se à vontade para questionar decisões ou ações dos que têm mais autoridade',
                '27. Nesta unidade, discutimos maneiras de impedir que os erros tornem a acontecer',
                '28. Os funcionários têm receio de perguntar quando algo não parece certo',
                '29. Quando acontece um erro, mas ele é identificado e corrigido antes de afetar o paciente, com que frequência é notificado?',
                '30. Quando acontece um erro, mas que não tem potencial de dano ao paciente, com que frequência é notificado?',
                '31. Quando acontece um erro que poderia prejudicar o paciente, mas isto não ocorreu, com que frequência é notificado?',
                '32. A administração do hospital cria um ambiente de trabalho que promove a segurança do paciente',
                '33. As unidades hospitalares não se coordenam bem entre si.',
                '34. Coisas "escapam por entre os dedos” quando os pacientes são transferidos de uma unidade para outra',
                '35. Existe uma boa cooperação entre as unidades hospitalares que precisam trabalhar juntas',
                '36. Importantes informações sobre a assistência se perdem durante as mudanças de turno',
                '37. Muitas vezes é desagradável trabalhar unidades com funcionários de outras unidades do hospital',
                '38. Os problemas com frequência ocorrem na troca de informações entre as unidades do hospital',
                '39. As ações da administração do hospital mostram que a segurança do paciente é uma prioridade máxima',
                '40. A administração do hospital parece se interessar pela segurança do paciente apenas quando acontece um evento adverso',
                '41. As unidades do hospital trabalham bem juntas para prestar o melhor atendimento aos pacientes',
                '42. As mudanças de turno são problemáticas para os pacientes deste Hospital']

In [6]:
colunas_adicionais = ['SEÇÃO G2: Notificou algum evento nos ultimos 12 meses',
                    'SEÇÃO G: Número de Eventos Notificados  Nos últimos 12 meses, quantas notificações de evento você preencheu e enviou?_0. Nenhuma notificações de eventos',
                    'SEÇÃO G: Número de Eventos Notificados  Nos últimos 12 meses, quantas notificações de evento você preencheu e enviou?_1. 1 a 2 notificações de eventos',
                    'SEÇÃO G: Número de Eventos Notificados  Nos últimos 12 meses, quantas notificações de evento você preencheu e enviou?_2. 3 a 5 notificações de eventos',
                    'SEÇÃO G: Número de Eventos Notificados  Nos últimos 12 meses, quantas notificações de evento você preencheu e enviou?_3. 6 a 10 notificações de eventos',
                    'SEÇÃO G: Número de Eventos Notificados  Nos últimos 12 meses, quantas notificações de evento você preencheu e enviou?_4. 11 a 20 notificações de eventos',
                    'SEÇÃO G: Número de Eventos Notificados  Nos últimos 12 meses, quantas notificações de evento você preencheu e enviou?_5. 21 notificações de eventos ou mais',
                    'SEÇÃO H: 3.2 Trabalha mais que 40horas semanais',
                    'SEÇÃO H: 3.2 Trabalha mais que 60horas semanais',
                    'SEÇÃO H: 3.2 Trabalha mais que 80horas semanais',
                    'Por favor, dê uma nota geral para a segurança do paciente em sua área/unidade de trabalho hospitalar._Excelente',
                    'Por favor, dê uma nota geral para a segurança do paciente em sua área/unidade de trabalho hospitalar._Muito boa',
                    'Por favor, dê uma nota geral para a segurança do paciente em sua área/unidade de trabalho hospitalar._Muito ruim',
                    'Por favor, dê uma nota geral para a segurança do paciente em sua área/unidade de trabalho hospitalar._Ruim',
                    'Por favor, dê uma nota geral para a segurança do paciente em sua área/unidade de trabalho hospitalar._Satisfatória',
                    'Nota Geral == Satisfatoria ?', 'Nota Geral == Boa ?',
                    'SEÇÃO H: 1.2 trabalha a mais de 1 ano no hospital?',
                    'SEÇÃO H: 1.3 trabalha a mais de 5 anos no hospital?',
                    'SEÇÃO H: 1. Há quanto tempo você trabalha neste hospital?_0. Menos de 1 ano',
                    'SEÇÃO H: 1. Há quanto tempo você trabalha neste hospital?_1. 1 a 5 anos',
                    'SEÇÃO H: 1. Há quanto tempo você trabalha neste hospital?_2. 6 a 10 anos',
                    'SEÇÃO H: 1. Há quanto tempo você trabalha neste hospital?_3. 11 a 15 anos',
                    'SEÇÃO H: 1. Há quanto tempo você trabalha neste hospital?_4. 16 a 20 anos',
                    'SEÇÃO H: 1. Há quanto tempo você trabalha neste hospital?_5. 21 anos ou mais',
                    '6. Há quanto tempo você trabalha em sua profissão ou especialização atual?_0. Menos de 1 ano',
                    '6. Há quanto tempo você trabalha em sua profissão ou especialização atual?_1. 1 a 5 anos',
                    '6. Há quanto tempo você trabalha em sua profissão ou especialização atual?_2. 6 a 10 anos',
                    '6. Há quanto tempo você trabalha em sua profissão ou especialização atual?_3. 11 a 15 anos',
                    '6. Há quanto tempo você trabalha em sua profissão ou especialização atual?_4. 16 a 20 anos',
                    '6. Há quanto tempo você trabalha em sua profissão ou especialização atual?_5. 21 anos ou mais']
lista_scores.extend(colunas_adicionais)
print(lista_scores)

['1. As pessoas se apoiam umas às outras nesta unidade', '2. Temos pessoas suficientes para lidar com o volume de trabalho', '3. Quando há muito trabalho a ser feito e rapidamente, trabalhamos juntos em equipe para realizar a tarefa', '4. Nesta unidade, as pessoas se tratam com respeito', '5. Os funcionários desta unidade trabalham mais horas do que o recomendado no atendimento a pacientes', '6. Estamos ativamente buscando melhorias para a segurança do paciente', '7. Utilizamos mais funcionários temporários do que o recomendado no atendimento a paciente', '8. Os funcionários sentem que seus erros são usados contra eles', '9. Erros que ocorreram levaram a mudanças positivas nesta unidade', '10. É apenas por acaso que erros mais sérios não acontecem por aqui', '11. Quando uma área nesta unidade fica muito movimentada, as demais ajudam', '12. Quando um evento é notificado, sentimos que o foco se concentra no indivíduo, e não no problema', '13. Quando fazemos mudanças para melhorar a segur

# Score por Ano x Acreditação

In [7]:
df = df_bruto.copy()

In [8]:
inicio = time.time() #iniciando o cronometro

resultado = pd.DataFrame()

for Ano in sorted(df['Ano'].unique()):

    ###################################################
    for acreditacao in range(2):
        dff = df[(df['Ano'] == Ano) & (df['acreditacao'] == acreditacao)].copy().reset_index(drop=True)
        fd = pd.DataFrame({'Ano': dff['Ano'].unique()})
        fd['acreditacao'] = acreditacao
        fd['quant'] = len(dff)
        fd['quant_unidade'] = len(dff['Unidade'].unique())

        for coluna in lista_scores:
            fd[coluna] = np.nan
            for i in range(len(fd)):
                if len(dff[coluna][dff['Ano'] == fd['Ano'][i]].value_counts(normalize=True)) > 1:
                    fd[coluna][i] = dff[coluna][dff['Ano'] == fd['Ano'][i]].value_counts(normalize=True)[1]*100
                else:
                    fd[coluna][i] = 0

        fd = fd.sort_values(by = 'Ano', ascending = True).reset_index(drop=True)
        resultado = pd.concat([resultado, fd]).reset_index(drop=True)

fim = time.time() #encerrando cronometro
print('Tempo para rodar a simulação:', round(fim - inicio, 3))

print('quantidade de linhas sumarizadas:', sum(resultado['quant']))

Tempo para rodar a simulação: 2.0
quantidade de linhas sumarizadas: 259268


In [9]:
df = resultado.copy()

df['Abertura de comunicação'] = 0
lista = ['24. Os funcionários falam voluntariamente se vêem algo que possa afetar negativamente o atendimento aos pacientes',
         '26. Os funcionários sentem-se à vontade para questionar decisões ou ações dos que têm mais autoridade',
         '28. Os funcionários têm receio de perguntar quando algo não parece certo']
for i in lista:
    df['Abertura de comunicação'] = df['Abertura de comunicação'] + df[i]
df['Abertura de comunicação'] = df['Abertura de comunicação']/len(lista)

df['Feedback e comunicação sobre erros'] = 0
lista = ['23. Recebemos feedback das mudanças implementadas com base nos eventos notificados',
         '25. Somos informados sobre os erros que acontecem nesta unidade',
         '27. Nesta unidade, discutimos maneiras de impedir que os erros tornem a acontecer']
for i in lista:
    df['Feedback e comunicação sobre erros'] = df['Feedback e comunicação sobre erros'] + df[i]
df['Feedback e comunicação sobre erros'] = df['Feedback e comunicação sobre erros']/len(lista)

df['Frequência de eventos relatados'] = 0
lista = ['29. Quando acontece um erro, mas ele é identificado e corrigido antes de afetar o paciente, com que frequência é notificado?',
         '30. Quando acontece um erro, mas que não tem potencial de dano ao paciente, com que frequência é notificado?',
         '31. Quando acontece um erro que poderia prejudicar o paciente, mas isto não ocorreu, com que frequência é notificado?']
for i in lista:
    df['Frequência de eventos relatados'] = df['Frequência de eventos relatados'] + df[i]
df['Frequência de eventos relatados'] = df['Frequência de eventos relatados']/len(lista)

df['Apoio de gestão hospitalar para segurança do paciente'] = 0
lista = ['32. A administração do hospital cria um ambiente de trabalho que promove a segurança do paciente',
         '39. As ações da administração do hospital mostram que a segurança do paciente é uma prioridade máxima',
         '40. A administração do hospital parece se interessar pela segurança do paciente apenas quando acontece um evento adverso']
for i in lista:
    df['Apoio de gestão hospitalar para segurança do paciente'] = df['Apoio de gestão hospitalar para segurança do paciente'] + df[i]
df['Apoio de gestão hospitalar para segurança do paciente'] = df['Apoio de gestão hospitalar para segurança do paciente']/len(lista)

df['Trabalho em equipe entre as unidades hospitalares'] = 0
lista = ['33. As unidades hospitalares não se coordenam bem entre si.',
         '35. Existe uma boa cooperação entre as unidades hospitalares que precisam trabalhar juntas',
         '37. Muitas vezes é desagradável trabalhar unidades com funcionários de outras unidades do hospital',
         '41. As unidades do hospital trabalham bem juntas para prestar o melhor atendimento aos pacientes']
for i in lista:
    df['Trabalho em equipe entre as unidades hospitalares'] = df['Trabalho em equipe entre as unidades hospitalares'] + df[i]
df['Trabalho em equipe entre as unidades hospitalares'] = df['Trabalho em equipe entre as unidades hospitalares']/len(lista)

df['Trabalho em equipe no âmbito das unidades'] = 0
lista = ['1. As pessoas se apoiam umas às outras nesta unidade',
         '3. Quando há muito trabalho a ser feito e rapidamente, trabalhamos juntos em equipe para realizar a tarefa',
         '4. Nesta unidade, as pessoas se tratam com respeito',
         '11. Quando uma área nesta unidade fica muito movimentada, as demais ajudam']
for i in lista:
    df['Trabalho em equipe no âmbito das unidades'] = df['Trabalho em equipe no âmbito das unidades'] + df[i]
df['Trabalho em equipe no âmbito das unidades'] = df['Trabalho em equipe no âmbito das unidades']/len(lista)

df['Transferências internas e passagens de plantão'] = 0
lista = ['34. Coisas "escapam por entre os dedos” quando os pacientes são transferidos de uma unidade para outra',
         '36. Importantes informações sobre a assistência se perdem durante as mudanças de turno',
         '38. Os problemas com frequência ocorrem na troca de informações entre as unidades do hospital',
         '42. As mudanças de turno são problemáticas para os pacientes deste Hospital']
for i in lista:
    df['Transferências internas e passagens de plantão'] = df['Transferências internas e passagens de plantão'] + df[i]
df['Transferências internas e passagens de plantão'] = df['Transferências internas e passagens de plantão']/len(lista)

df['Aprendizado organizacional - melhoria contínua'] = 0
lista = ['6. Estamos ativamente buscando melhorias para a segurança do paciente',
         '9. Erros que ocorreram levaram a mudanças positivas nesta unidade',
         '13. Quando fazemos mudanças para melhorar a segurança do paciente, nós avaliamos sua efetividade']
for i in lista:
    df['Aprendizado organizacional - melhoria contínua'] = df['Aprendizado organizacional - melhoria contínua'] + df[i]
df['Aprendizado organizacional - melhoria contínua'] = df['Aprendizado organizacional - melhoria contínua']/len(lista)

df['Expectativas de promoção da segurança dos supervisores e gerentes'] = 0
lista = ['19. Meu supervisor/gerente elogia quando vê um trabalho feito de acordo com os procedimentos estabelecidos para a segurança do paciente.',
         '20. Meu supervisor/gerente considera seriamente as sugestões dos funcionários para melhorar a segurança do paciente',
         '21. Sempre que a pressão aumenta, meu supervisor/gerente quer que trabalhemos mais rápido, mesmo que isto signifique tomar atalhos',
         '22. Meu supervisor/gerente ignora problemas recorrentes na segurança do paciente']
for i in lista:
    df['Expectativas de promoção da segurança dos supervisores e gerentes'] = df['Expectativas de promoção da segurança dos supervisores e gerentes'] + df[i]
df['Expectativas de promoção da segurança dos supervisores e gerentes'] = df['Expectativas de promoção da segurança dos supervisores e gerentes']/len(lista)

df['Percepção gerais sobre segurança'] = 0
lista = ['10. É apenas por acaso que erros mais sérios não acontecem por aqui',
         '15. A segurança do paciente nunca é sacrificada em prol de se trabalhar mais',
         '17. Temos problemas com a segurança do paciente nesta unidade',
         '18. Nossos procedimentos e sistemas são bons para impedir que os erros aconteçam']
for i in lista:
    df['Percepção gerais sobre segurança'] = df['Percepção gerais sobre segurança'] + df[i]
df['Percepção gerais sobre segurança'] = df['Percepção gerais sobre segurança']/len(lista)

df['Respostas não punitivas aos erros'] = 0
lista = ['8. Os funcionários sentem que seus erros são usados contra eles',
         '12. Quando um evento é notificado, sentimos que o foco se concentra no indivíduo, e não no problema',
         '16. Os funcionários se preocupam que seus erros sejam registrados em seu arquivo funcional']
for i in lista:
    df['Respostas não punitivas aos erros'] = df['Respostas não punitivas aos erros'] + df[i]
df['Respostas não punitivas aos erros'] = df['Respostas não punitivas aos erros']/len(lista)

df['Staffing'] = 0
lista = ['2. Temos pessoas suficientes para lidar com o volume de trabalho',
         '5. Os funcionários desta unidade trabalham mais horas do que o recomendado no atendimento a pacientes',
         '7. Utilizamos mais funcionários temporários do que o recomendado no atendimento a paciente',
         '14. Trabalhamos “em modo de crise”, tentando fazer coisas demais, rápido demais']
for i in lista:
    df['Staffing'] = df['Staffing'] + df[i]
df['Staffing'] = df['Staffing']/len(lista)

In [10]:
display(df)

,Ano,acreditacao,quant,quant_unidade,1. As pessoas se apoiam umas às outras nesta unidade,2. Temos pessoas suficientes para lidar com o volume de trabalho,"3. Quando há muito trabalho a ser feito e rapidamente, trabalhamos juntos em equipe para realizar a tarefa","4. Nesta unidade, as pessoas se tratam com respeito",5. Os funcionários desta unidade trabalham mais horas do que o recomendado no atendimento a pacientes,6. Estamos ativamente buscando melhorias para a segurança do paciente,...,Frequência de eventos relatados,Apoio de gestão hospitalar para segurança do paciente,Trabalho em equipe entre as unidades hospitalares,Trabalho em equipe no âmbito das unidades,Transferências internas e passagens de plantão,Aprendizado organizacional - melhoria contínua,Expectativas de promoção da segurança dos supervisores e gerentes,Percepção gerais sobre segurança,Respostas não punitivas aos erros,Staffing
0,2014,0,366,2,64.088398,34.444444,78.787879,82.960894,44.972067,80.769231,...,64.401727,62.899690,48.931344,67.841974,45.662204,72.525345,66.335835,52.832017,36.685189,45.970429
1,2014,1,6483,10,59.896406,27.621723,72.874747,75.331565,51.119812,81.624667,...,54.090129,61.514487,47.614657,62.432830,40.388309,69.637063,63.841662,52.092022,38.008854,43.997124
2,2015,0,756,3,66.351351,52.069426,85.885486,83.266399,65.369128,89.558233,...,62.991608,75.478086,61.605665,72.656133,53.764626,77.545146,71.924208,61.793261,41.228767,61.586276
3,2015,1,12562,21,62.507063,33.069659,75.924583,77.408691,51.437596,85.657661,...,58.433697,68.284922,53.719786,65.714363,45.658074,74.854024,67.591973,56.942783,40.051830,47.080484
4,2016,0,1377,3,66.372981,43.452816,85.212299,82.564103,60.528247,92.668622,...,61.449900,80.503428,65.305091,72.830903,57.459563,82.534409,75.599338,64.882052,39.665973,56.153138
5,2016,1,22129,26,64.550265,40.657187,78.931548,80.349166,54.748399,89.173867,...,60.966861,74.320213,59.594116,68.854543,51.062665,78.998655,70.284771,61.107103,39.711183,52.025357
6,2017,0,2628,6,66.897081,49.138921,83.766234,83.058643,60.899654,90.053763,...,62.318576,77.024249,65.623150,73.255624,56.992745,79.966291,74.909938,63.354432,39.912583,57.907077
7,2017,1,24070,28,65.949144,40.860486,79.252869,81.338087,56.335480,90.156967,...,62.799171,76.410659,61.173927,69.545630,52.981856,79.835931,71.305999,62.651628,40.468809,52.657720
8,2018,0,4444,8,67.661125,41.191123,79.995464,80.675890,54.292237,88.465922,...,59.616445,74.462434,60.766780,70.745511,53.215630,77.810124,72.308685,61.066727,39.635050,52.416235
9,2018,1,23384,27,65.557884,36.621132,77.927515,80.562347,53.337416,89.184227,...,61.353977,74.317648,59.479429,68.815316,50.963605,78.984200,70.030549,61.570372,40.503813,49.899055


In [11]:
df.to_excel('3 - Dados_Score_Ano_vs_Acreditacao.xlsx')

In [12]:
stop

NameError: name 'stop' is not defined

# Score Por Ano x Acreditação x Unidade

In [ ]:
df = df_bruto.copy()

In [ ]:
inicio = time.time() #iniciando o cronometro

resultado = pd.DataFrame()

for Ano in sorted(df['Ano'].unique()):

    ###################################################
    for acreditacao in range(2):
        dff = df[(df['Ano'] == Ano) & (df['acreditacao'] == acreditacao)].copy().reset_index(drop=True)
        fd = pd.DataFrame({'Unidade': dff['Unidade'].unique()})
        fd['acreditacao'] = acreditacao
        
        fd['Acreditação'] = np.nan
        fd['Acreditação2'] = np.nan
        
        fd['Ano'] = Ano
        fd['Regional'] = np.nan
        fd['Estado'] = np.nan
        fd['quant'] = np.nan

        for coluna in lista_scores:
            fd[coluna] = np.nan
            for i in range(len(fd)):
                if len(dff[coluna][dff['Unidade'] == fd['Unidade'][i]].value_counts(normalize=True)) > 1:
                    fd[coluna][i] = dff[coluna][dff['Unidade'] == fd['Unidade'][i]].value_counts(normalize=True)[1]*100
                else:
                    fd[coluna][i] = 0
                fd['Regional'][i] = dff['Regional'][dff['Unidade'] == fd['Unidade'][i]].values[0]
                fd['Estado'][i] = dff['Estado'][dff['Unidade'] == fd['Unidade'][i]].values[0]
                fd['quant'][i] = len(dff[dff['Unidade'] == fd['Unidade'][i]])
                
                fd['Acreditação'][i] = dff['Acreditação'][(dff['Unidade'] == fd['Unidade'][i]) &
                                                          (dff['Ano'] == fd['Ano'][i])].values[0]
                fd['Acreditação2'][i] = dff['Acreditação2'][(dff['Unidade'] == fd['Unidade'][i]) &
                                                            (dff['Ano'] == fd['Ano'][i])].values[0]
                
        resultado = pd.concat([resultado, fd]).reset_index(drop=True)

fim = time.time() #encerrando cronometro
print('Tempo para rodar a simulação:', round(fim - inicio, 3))

print('quantidade de linhas sumarizadas:', sum(resultado['quant']))
df = resultado.sort_values(by = ['Unidade','Ano'], ascending = True).reset_index(drop=True)

In [ ]:
df['Abertura de comunicação'] = 0
lista = ['24. Os funcionários falam voluntariamente se vêem algo que possa afetar negativamente o atendimento aos pacientes',
         '26. Os funcionários sentem-se à vontade para questionar decisões ou ações dos que têm mais autoridade',
         '28. Os funcionários têm receio de perguntar quando algo não parece certo']
for i in lista:
    df['Abertura de comunicação'] = df['Abertura de comunicação'] + df[i]
df['Abertura de comunicação'] = df['Abertura de comunicação']/len(lista)

df['Feedback e comunicação sobre erros'] = 0
lista = ['23. Recebemos feedback das mudanças implementadas com base nos eventos notificados',
         '25. Somos informados sobre os erros que acontecem nesta unidade',
         '27. Nesta unidade, discutimos maneiras de impedir que os erros tornem a acontecer']
for i in lista:
    df['Feedback e comunicação sobre erros'] = df['Feedback e comunicação sobre erros'] + df[i]
df['Feedback e comunicação sobre erros'] = df['Feedback e comunicação sobre erros']/len(lista)

df['Frequência de eventos relatados'] = 0
lista = ['29. Quando acontece um erro, mas ele é identificado e corrigido antes de afetar o paciente, com que frequência é notificado?',
         '30. Quando acontece um erro, mas que não tem potencial de dano ao paciente, com que frequência é notificado?',
         '31. Quando acontece um erro que poderia prejudicar o paciente, mas isto não ocorreu, com que frequência é notificado?']
for i in lista:
    df['Frequência de eventos relatados'] = df['Frequência de eventos relatados'] + df[i]
df['Frequência de eventos relatados'] = df['Frequência de eventos relatados']/len(lista)

df['Apoio de gestão hospitalar para segurança do paciente'] = 0
lista = ['32. A administração do hospital cria um ambiente de trabalho que promove a segurança do paciente',
         '39. As ações da administração do hospital mostram que a segurança do paciente é uma prioridade máxima',
         '40. A administração do hospital parece se interessar pela segurança do paciente apenas quando acontece um evento adverso']
for i in lista:
    df['Apoio de gestão hospitalar para segurança do paciente'] = df['Apoio de gestão hospitalar para segurança do paciente'] + df[i]
df['Apoio de gestão hospitalar para segurança do paciente'] = df['Apoio de gestão hospitalar para segurança do paciente']/len(lista)

df['Trabalho em equipe entre as unidades hospitalares'] = 0
lista = ['33. As unidades hospitalares não se coordenam bem entre si.',
         '35. Existe uma boa cooperação entre as unidades hospitalares que precisam trabalhar juntas',
         '37. Muitas vezes é desagradável trabalhar unidades com funcionários de outras unidades do hospital',
         '41. As unidades do hospital trabalham bem juntas para prestar o melhor atendimento aos pacientes']
for i in lista:
    df['Trabalho em equipe entre as unidades hospitalares'] = df['Trabalho em equipe entre as unidades hospitalares'] + df[i]
df['Trabalho em equipe entre as unidades hospitalares'] = df['Trabalho em equipe entre as unidades hospitalares']/len(lista)

df['Trabalho em equipe no âmbito das unidades'] = 0
lista = ['1. As pessoas se apoiam umas às outras nesta unidade',
         '3. Quando há muito trabalho a ser feito e rapidamente, trabalhamos juntos em equipe para realizar a tarefa',
         '4. Nesta unidade, as pessoas se tratam com respeito',
         '11. Quando uma área nesta unidade fica muito movimentada, as demais ajudam']
for i in lista:
    df['Trabalho em equipe no âmbito das unidades'] = df['Trabalho em equipe no âmbito das unidades'] + df[i]
df['Trabalho em equipe no âmbito das unidades'] = df['Trabalho em equipe no âmbito das unidades']/len(lista)

df['Transferências internas e passagens de plantão'] = 0
lista = ['34. Coisas "escapam por entre os dedos” quando os pacientes são transferidos de uma unidade para outra',
         '36. Importantes informações sobre a assistência se perdem durante as mudanças de turno',
         '38. Os problemas com frequência ocorrem na troca de informações entre as unidades do hospital',
         '42. As mudanças de turno são problemáticas para os pacientes deste Hospital']
for i in lista:
    df['Transferências internas e passagens de plantão'] = df['Transferências internas e passagens de plantão'] + df[i]
df['Transferências internas e passagens de plantão'] = df['Transferências internas e passagens de plantão']/len(lista)

df['Aprendizado organizacional - melhoria contínua'] = 0
lista = ['6. Estamos ativamente buscando melhorias para a segurança do paciente',
         '9. Erros que ocorreram levaram a mudanças positivas nesta unidade',
         '13. Quando fazemos mudanças para melhorar a segurança do paciente, nós avaliamos sua efetividade']
for i in lista:
    df['Aprendizado organizacional - melhoria contínua'] = df['Aprendizado organizacional - melhoria contínua'] + df[i]
df['Aprendizado organizacional - melhoria contínua'] = df['Aprendizado organizacional - melhoria contínua']/len(lista)

df['Expectativas de promoção da segurança dos supervisores e gerentes'] = 0
lista = ['19. Meu supervisor/gerente elogia quando vê um trabalho feito de acordo com os procedimentos estabelecidos para a segurança do paciente.',
         '20. Meu supervisor/gerente considera seriamente as sugestões dos funcionários para melhorar a segurança do paciente',
         '21. Sempre que a pressão aumenta, meu supervisor/gerente quer que trabalhemos mais rápido, mesmo que isto signifique tomar atalhos',
         '22. Meu supervisor/gerente ignora problemas recorrentes na segurança do paciente']
for i in lista:
    df['Expectativas de promoção da segurança dos supervisores e gerentes'] = df['Expectativas de promoção da segurança dos supervisores e gerentes'] + df[i]
df['Expectativas de promoção da segurança dos supervisores e gerentes'] = df['Expectativas de promoção da segurança dos supervisores e gerentes']/len(lista)

df['Percepção gerais sobre segurança'] = 0
lista = ['10. É apenas por acaso que erros mais sérios não acontecem por aqui',
         '15. A segurança do paciente nunca é sacrificada em prol de se trabalhar mais',
         '17. Temos problemas com a segurança do paciente nesta unidade',
         '18. Nossos procedimentos e sistemas são bons para impedir que os erros aconteçam']
for i in lista:
    df['Percepção gerais sobre segurança'] = df['Percepção gerais sobre segurança'] + df[i]
df['Percepção gerais sobre segurança'] = df['Percepção gerais sobre segurança']/len(lista)

df['Respostas não punitivas aos erros'] = 0
lista = ['8. Os funcionários sentem que seus erros são usados contra eles',
         '12. Quando um evento é notificado, sentimos que o foco se concentra no indivíduo, e não no problema',
         '16. Os funcionários se preocupam que seus erros sejam registrados em seu arquivo funcional']
for i in lista:
    df['Respostas não punitivas aos erros'] = df['Respostas não punitivas aos erros'] + df[i]
df['Respostas não punitivas aos erros'] = df['Respostas não punitivas aos erros']/len(lista)

df['Staffing'] = 0
lista = ['2. Temos pessoas suficientes para lidar com o volume de trabalho',
         '5. Os funcionários desta unidade trabalham mais horas do que o recomendado no atendimento a pacientes',
         '7. Utilizamos mais funcionários temporários do que o recomendado no atendimento a paciente',
         '14. Trabalhamos “em modo de crise”, tentando fazer coisas demais, rápido demais']
for i in lista:
    df['Staffing'] = df['Staffing'] + df[i]
df['Staffing'] = df['Staffing']/len(lista)

In [ ]:
df['passou_a_ser_acreditado'] = 0
tabela = pd.DataFrame(columns = df.columns)
count = 0
for unidade in sorted(df['Unidade'].unique()):
    if len(df['acreditacao'][df['Unidade'] == unidade].unique()) >= 2:
        df['passou_a_ser_acreditado'][df['Unidade'] == unidade] = 1
        print(unidade, '='*25)
        print(df[df['Unidade'] == unidade].groupby(['Ano', 'acreditacao']).size(), '\n')
        count += 1
        
        tabela = pd.concat([tabela, df[df['Unidade'] == unidade]])
        
print('='*25, 'QUANTIDADE:', count, '='*25)
display(tabela)

display(df['passou_a_ser_acreditado'].value_counts())

In [ ]:
for i in df['passou_a_ser_acreditado'].unique():
    print(i,'='*50)
    count = 0
    for t in df['Unidade'][df['passou_a_ser_acreditado'] == i].unique():
        print(df['acreditacao'][df['Unidade'] == t].unique())
        count += 1
    print(count)

df['passou_a_ser_acreditado'].value_counts()

In [ ]:
df['status_eterno'] = np.nan

df['status_eterno'][df['passou_a_ser_acreditado'] == 0] = df['acreditacao']
df['status_eterno'].value_counts()

In [ ]:
display(pd.concat([df.head(10),df.tail(10)]))

In [ ]:
df.to_excel('4 - Dados_Score_Ano_vs_Acreditacao_vs_Unidade.xlsx')